In [11]:
from my_import import *
df_train = pd.read_csv('df_train.csv')
df_val = pd.read_csv('df_val.csv')
df_test = pd.read_csv('df_test.csv')
df_full = pd.read_csv('final_cleaned_dataset_df.csv')

#Make sure the genre collumns is in lists not strings
#NEED TO DO THIS EVERYTIME EXPORT DATASET
df_train['genres'] = df_train['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_val['genres'] = df_val['genres'].apply(lambda x: list(ast.literal_eval(x)))
df_test['genres'] = df_test['genres'].apply(lambda x: list(ast.literal_eval(x)))

df_train=df_train.drop(columns=['title','index'])
df_val=df_val.drop(columns=['title','index'])
df_test=df_test.drop(columns=['title','index'])


display(df_train)


,synopsis,genres
0,Shuramaru is hated and feared by the villagers...,[Supernatural]
1,"Sometime in the future, the world was complete...","[Action, Adventure]"
2,"Set in 2014, the anime follows the adventures ...",[Comedy]
3,"This story is about Mick, a sleeping chironomi...",[Adventure]
4,The anime is based on MegaHouse's line of Zigu...,"[Action, Mecha]"
...,...,...
7115,Picture Drama episodes included in each DVD vo...,"[Action, Adventure, Comedy, Drama, Fantasy, Ro..."
7116,"On a hot summer day, Takashi Natsume and his f...","[Drama, Shoujo, Slice of Life, Supernatural]"
7117,"Due to the arrival of aliens called the ""Amant...","[Action, Comedy, Historical, Mecha, Sci-Fi, Sh..."
7118,The Konohagakure Grand Sports Festival has beg...,"[Action, Comedy, Fantasy, Shounen, Sports]"


In [12]:
from transformers import pipeline
import torch

# Check for GPU availability
device = 0 if torch.cuda.is_available() else -1

# Load the zero-shot classification model
classifier = pipeline("zero-shot-classification", 
                      model="facebook/bart-large-mnli", 
                      device=device)



Device set to use cuda:0


In [22]:
def predict_genres(text, candidate_labels, threshold=0.5):
    result = classifier(text, candidate_labels=candidate_labels, multi_label=True)
    # Filter genres based on threshold
    print(text)
    print("Predicted Genres:")
    for label, score in zip(result['labels'], result['scores']):
        print(f"{label}: {score:.4f}")
    predicted_genres = [label for label, score in zip(result['labels'], result['scores']) if score >= threshold]
    return predicted_genres


In [23]:
# Get all unique genres from your training data as candidate labels
unique_genres = sorted(set(genre for genres in df_train['genres'] for genre in genres))

# Predict genres for each synopsis in the test set
df_test['predicted_genres'] = df_test['synopsis'].apply(lambda x: predict_genres(x, unique_genres))

# Display the first few results
display(df_test[['synopsis', 'predicted_genres']].head())


The daily life of the Motsumoto family. The three sisters, Fuu, Suu, and Chii, follow what happens on a daily basis. Mother and Father have to deal with them sometimes, but they're used to it.
Predicted Genres:
Action: 0.1585
Slice of Life: 0.1535
Drama: 0.0332
Adventure: 0.0310
Super Power: 0.0264
Mystery: 0.0214
Shounen: 0.0214
Historical: 0.0194
Shoujo: 0.0188
Kids: 0.0146
Seinen: 0.0128
Romance: 0.0113
School: 0.0092
Mecha: 0.0089
Music: 0.0084
Comedy: 0.0059
Sports: 0.0049
Supernatural: 0.0037
Sci-Fi: 0.0025
Fantasy: 0.0006
The Soul Tree, the great source of our race.  But for its ultimate power and by those whose minds were blinded by it, it also became the catastrophic cause that had brought never-ending wars between the tribes. Tortured by the agonies of the wars, the leaders of the tribes finally came to a peace agreement that they would instead open a competition. Players, equipped with special watches and shields, compete against each other as winner’s tribe would be awarded

,synopsis,predicted_genres
0,The daily life of the Motsumoto family. The th...,[]
1,"The Soul Tree, the great source of our race. ...",[Historical]
2,"Fairies living in a fluffy forest, where both ...",[Fantasy]
3,A cyborg warrior from an ancient Antarctic kin...,"[Adventure, Historical, Sci-Fi, Fantasy, Actio..."
4,The 2018 LINE sticker set Poccolies is inspiri...,[]


In [25]:
# Display the first few results
display(df_test[['synopsis', 'genres','predicted_genres']])

,synopsis,genres,predicted_genres
0,The daily life of the Motsumoto family. The th...,"[Comedy, Slice of Life]",[]
1,"The Soul Tree, the great source of our race. ...","[Action, Adventure, Kids]",[Historical]
2,"Fairies living in a fluffy forest, where both ...","[Fantasy, Kids]",[Fantasy]
3,A cyborg warrior from an ancient Antarctic kin...,"[Action, Mecha, Sci-Fi]","[Adventure, Historical, Sci-Fi, Fantasy, Actio..."
4,The 2018 LINE sticker set Poccolies is inspiri...,"[Kids, Slice of Life]",[]
...,...,...,...
886,A series of four animated prequels to Street F...,[Action],[]
887,"According to HMV, the DVD & Blu-ray volume 7 o...","[Comedy, Seinen, Sports]",[]
888,"Lucius ends up in a space station, where he ex...","[Comedy, Seinen]","[Sci-Fi, Adventure]"
889,"The ""Megumi and Taiyou"" commercial depicts the...","[Romance, Slice of Life]","[Historical, Romance]"


In [ ]:
df_test.to_csv('predicted_genres_test.csv', index=False)
print("Predictions saved to 'predicted_genres_test.csv'")


In [21]:
synopsis = ["This story is about Mick, a sleeping chironomid. Insects are taken up into space for use in experiments by humans. An epic space adventure of Mick and his friends of their journey back home to Earth. The development plans and experiments are aborted by humans for no known reason, the organisms are left behind in the space station with limited resources. Going over hurdles and fighting through challenges, together, they travel across planets and gain new comrades, heading for Earth. As their journey progresses, the mystery behind the human’s abandonment unravels…"]
for i in range(0,len(synopsis)):
    result = classifier(
        synopsis[i],
        candidate_labels=unique_genres,
        multi_label=True  # Set to True for multi-label classification
    )
    print(synopsis[i])
    print("Predicted Genres:")
    for label, score in zip(result['labels'], result['scores']):
        print(f"{label}: {score:.4f}")




This story is about Mick, a sleeping chironomid. Insects are taken up into space for use in experiments by humans. An epic space adventure of Mick and his friends of their journey back home to Earth. The development plans and experiments are aborted by humans for no known reason, the organisms are left behind in the space station with limited resources. Going over hurdles and fighting through challenges, together, they travel across planets and gain new comrades, heading for Earth. As their journey progresses, the mystery behind the human’s abandonment unravels…
Predicted Genres:
Adventure: 0.9848
Sci-Fi: 0.8690
Mystery: 0.7716
Fantasy: 0.6724
Drama: 0.5897
Historical: 0.4351
Action: 0.3016
Slice of Life: 0.2499
Kids: 0.0763
Seinen: 0.0702
Super Power: 0.0649
Comedy: 0.0562
School: 0.0398
Shoujo: 0.0371
Romance: 0.0361
Shounen: 0.0267
Supernatural: 0.0260
Music: 0.0051
Mecha: 0.0032
Sports: 0.0026
